In [101]:
import pandas as pd
import numpy

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

#Import data
all_colors = pd.read_csv("data/all-colors.csv")
chosen_colors = pd.read_csv("data/chosen-colors.csv")

#Remove chosen colors
condition = all_colors["Shirt Color Name"].isin(chosen_colors["Shirt Color"])
remaining_colors = all_colors.drop(all_colors[condition].index, inplace=False)

#Confirm removal
# print(len(remaining_colors))
# print(len(all_colors))
# print(len(chosen_colors))

In [108]:
import colormath.color_objects as co
import colormath.color_diff as cdiff

def compute_average_distance(row):
    color_Lab = None

    color_Lab = co.LabColor(*map(float, row["L*a*b Value"].split(", ")))

    distances = []

    #Compute all distances from chosen colors
    for index, row_remaining in chosen_colors.iterrows():
        other_color = co.LabColor(*map(float, row_remaining["L*a*b Value"].split(", ")))

        #Newer, asymmetric formula, weighted for fabrics, using
        #distance = cdiff.delta_e_cmc(other_color, color_Lab , pl=2, pc=1)

        #Older symmetric formula, weighted for fabrics
        distance = cdiff.delta_e_cie2000(color_Lab, other_color,Kl=2, Kc=1, Kh=1)

        distances += [distance]

    average_distance = sum(distances)/len(remaining_colors)

    row["Distance"] = average_distance
    return row

remaining_colors = remaining_colors.apply(compute_average_distance, axis=1)

In [109]:
sorted_remaining = remaining_colors.sort_values(by = "Distance", ascending=False)
sorted_remaining.iloc[:].head(10)

,Shirt Color Name,PMS Color Code,L*a*b Value,Hex,Distance
47,Teal,3262C,"66.44, -59.33, -12.05",00BFB2,7.545582
73,Heather Kelly,340 C,"51.80, -65.36, 14.55",00965E,7.436395
50,Turquoise,638C,"63.99, -33.79, -34.95",00AFD7,7.312165
24,Maize Yellow,7548CP,"83.14, 5.01, 85.02",F2CA00,7.287751
104,Heather Yellow,100 CP,"92.09, -6.83, 62.42",F6EB69,7.094500
96,Heather Sea Green,2398 C,"66.81, -46.18, -10.83",00BAB3,6.989276
37,Poppy,2347C,"49.45, 75.12, 67.21",E10600,6.858418
20,Kelly,341C,"43.23, -48.36, 9.96",007A53,6.852545
83,Heather Orange,1645 C,"65.69, 58.38, 58.65",FF6A39,6.630476
34,Orange,1645C,"65.69, 58.38, 58.65",FF6A39,6.630476


In [111]:
chosen_colors.to_json(orient="records")

'[{"Shirt Color":"Heather Ice Blue","Pantone Color":"317 CP","L*a*b Value":"85.65, -11.38, -2.20","HEX":"BDDDD9"},{"Shirt Color":"Berry","Pantone Color":"233 CP","L*a*b Value":"45.38, 71.84, -7.30","HEX":"C6057B"},{"Shirt Color":"Oxblood Black","Pantone Color":"7449 CP","L*a*b Value":"28.13, 16.23, -6.49","HEX":"55394D"},{"Shirt Color":"White","Pantone Color":"White C","L*a*b Value":"100, 0, 0","HEX":"FFFFFF"},{"Shirt Color":"Ash","Pantone Color":"428 CP","L*a*b Value":"78.50, -2.14, -0.31","HEX":"BDC4C2"},{"Shirt Color":"Gold","Pantone Color":"137 C","L*a*b Value":"76.47, 31.14, 81.64","HEX":"FFA400"},{"Shirt Color":"Yellow","Pantone Color":"100 CP","L*a*b Value":"92.09, -6.83, 62.42","HEX":"F6EB69"},{"Shirt Color":"Deep Teal","Pantone Color":"3025 CP","L*a*b Value":"32.45, -23.19, -27.24","HEX":"005876"},{"Shirt Color":"Heather Maroon","Pantone Color":"7644 CP","L*a*b Value":"27.76, 25.21, -2.86","HEX":"603147"},{"Shirt Color":"Evergreen","Pantone Color":"343 C","L*a*b Value":"31.90,